In [3]:
import pandas as pd

In [8]:
def clean_data(raw_path):
    """
    Concatenate raw dataframes with <s> token indicating the start of the collection and save the series of executables into dataframe

    Parameters:
    raw_path (str): path of raw dataframes
    """
    datasets = []
    # start = pd.Series(['<s>'])
    for i in range(1, 30):
        if i == 28:
            continue
        file_path = f"{raw_path}/df{i}.csv"
        df = pd.read_csv(file_path)
        temp = df[df['ID_INPUT'] == 3][['MEASUREMENT_TIME', 'VALUE']].reset_index(drop=True)
        temp = temp.rename(columns={'MEASUREMENT_TIME': 'Start', 'VALUE': 'Value'})
        temp['Start'] = pd.to_datetime(temp['Start']).dt.tz_localize(tz='GMT+0').dt.tz_convert('America/Los_Angeles').dt.tz_localize(None)
        temp['End'] = temp['Start'].shift(-1)
        temp['Duration'] = (temp['Start'].shift(-1) - temp['Start'])
        temp = temp.drop(len(temp)-1)
        datasets.append(temp)
    output_dataset = pd.concat(datasets).reset_index(drop=True)
    # print(output_dataset['Duration'].dtype)
    output_dataset.to_csv('/Users/yikaimao/Desktop/DSC_180B/Intel-capstone/data/processed/lstm_dataset_local.csv', index=False)
    return output_dataset

In [9]:
lstm_dataset = clean_data('/Users/yikaimao/Desktop/DSC_180B/Intel-capstone/data/raw')
lstm_dataset

,Start,Value,End,Duration
0,2022-12-12 12:34:09.893,VsDebugConsole.exe,2022-12-12 12:34:12.896,0 days 00:00:03.003000
1,2022-12-12 12:34:12.896,firefox.exe,2022-12-12 12:52:55.024,0 days 00:18:42.128000
2,2022-12-12 12:52:55.024,VsDebugConsole.exe,2022-12-12 17:06:02.811,0 days 04:13:07.787000
3,2022-12-12 17:06:02.811,explorer.exe,2022-12-12 17:06:14.814,0 days 00:00:12.003000
4,2022-12-12 17:06:14.814,VsDebugConsole.exe,2022-12-12 17:06:56.818,0 days 00:00:42.004000
...,...,...,...,...
4856,2023-01-13 01:20:13.327,League of Legends.exe,2023-01-13 01:29:11.877,0 days 00:08:58.550000
4857,2023-01-13 01:29:11.877,LeagueClientUx.exe,2023-01-13 01:52:43.174,0 days 00:23:31.297000
4858,2023-01-13 01:52:43.174,firefox.exe,2023-01-13 02:23:45.919,0 days 00:31:02.745000
4859,2023-01-13 02:23:45.919,LeagueClientUx.exe,2023-01-13 02:23:49.144,0 days 00:00:03.225000


In [90]:
lstm_dataset.groupby([lstm_dataset.Start.dt.date, lstm_dataset.Value]).sum()

Duration
Start      Value                                         
2022-12-12 Discord.exe             0 days 00:03:03.020000
           League of Legends.exe   0 days 00:43:48.148000
           LeagueClientUx.exe      0 days 00:21:04.029000
           SearchApp.exe           0 days 00:00:51.006000
           ShellExperienceHost.exe 0 days 00:00:09.001000
...                                                   ...
2023-01-12 explorer.exe            0 days 00:00:17.187000
           firefox.exe             0 days 00:43:19.285000
2023-01-13 League of Legends.exe   0 days 01:10:43.933000
           LeagueClientUx.exe      0 days 00:34:58.930000
           firefox.exe             0 days 00:40:17.345000

[251 rows x 1 columns]

In [93]:
lstm_dataset['Value'].unique()

array(['VsDebugConsole.exe', 'firefox.exe', 'explorer.exe', 'Teams.exe',
       'SearchApp.exe', 'LeagueClientUx.exe', 'Discord.exe',
       'notepad.exe', 'League of Legends.exe', 'ShellExperienceHost.exe',
       'vpnui.exe', 'vpndownloader.exe', 'notepad++.exe', 'cmd.exe',
       'devenv.exe', 'steam.exe', 'steamwebhelper.exe', 'Yakuza0.exe',
       'chrome.exe', 'Zoom.exe', 'Talkdesk.exe', 'ScreenClippingHost.exe',
       'msedge.exe', 'OUTLOOK.EXE', 'obs64.exe', 'vlc.exe', 'slack.exe',
       'WeChat.exe', 'RiotClientUx.exe', 'LeagueClient.exe',
       'LeagueCrashHandler.exe', 'audacity.exe', '7zFM.exe',
       'DB Browser for SQLite.exe', 'Shell_TrayWnd', 'Chrome_WidgetWin_1',
       'MSTaskListWClass', 'MozillaWindowClass', 'ToolbarWindow32',
       'Windows.UI.Core.CoreWindow', 'Button', '#32768', 'DirectUIHWND',
       '#32770', 'StartMenuExperienceHost.exe', 'WindowsCamera.exe',
       'ApplicationFrameHost.exe', 'iCloud.exe', 'iCloudServices.exe',
       'Taskmgr.exe', 'Mic